# Lab XX: An Introduction to Clustering

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd, numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.spatial.distance import cdist

## There are two main groups of activities you can do with Clustering

If you are interested in using K-Means to describe your (unlabeled) data, you can use Kmeans to split your data into $k$ clusters and then analyze the clusters. This is what we are going to do in this notebook.

You can also use a similar process called K-nearest neighbours (KNN) for classification tasks. In KNN, clusters are formed with **labeled** data and then **unlabeled** data is categorized into the clusters.

The good news is that although K-Means is a machine learning algorithm, it is reletively simple to implement. You don't have to be familiar with the math behind K-Means, but a conceptual demonstration of the algorithm from UC Berkeley's Data 100 lecture on clustering is available [here](lec23_clustering_d100_excerpt.pdf). 

A guide on the algorithm is also documented [here](https://scikit-learn.org/stable/modules/clustering.html#k-means) by `scikit-learn`. 

### The Data
For the purposes of this exercise, we are going to be using a dataset of transit stops in Toronto, along with census variables aggregated from the census tract level to the neighbourhood in which the stops are located.

It was created in the `create_stops_data.ipynb` notebook, which you are welcome to review but you do not need to recreate it. 

In [ ]:
stops = pd.read_csv("toronto_stops.csv").drop(columns = ["Unnamed: 0"])
stops.head()

The K-Means algorithm only works with numeric data (floats and ints, in Python). Omit the columns with the stop id and neighbourhood name so that the data being put into the algorithm only consists of the numeric stop charactaristics.

You might notice that a lot of the rows have the same values for the demographic variables. That is because the demographic variables averaged at the neighbourhood level- and there are many transit stops in the same neighbourhood.

In [ ]:
X = stops.iloc[:, 2:]
X.head()

In [ ]:
X.describe()

This code below creates a `KMeans` object with $k$ clusters (`n_clusters`). This notebook was written using `n_clusters = 4`, but you are welcome to change this variable to 3, 5, 6, 7-- whatever seems like a reasonable number of distinct clusters in Toronto based on the variables selected. The data `X` is then fitted to the Kmeans object, which generates a set of 9,027 labels- one label for each stop!

In [ ]:
# try running the notebook with various values for n_clusters
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

Make a column called `cluster` in the `stops` dataset to see the labeled data. There are $k$ unique entries in the `y_kmeans` array, one for each cluster that you specified above. It makes a bit more sense to think about the clusters as beginning with cluster 1 instead of cluster 0, so let's shift the array by 1.

In [ ]:
np.unique(y_kmeans+1)

In [ ]:
stops["cluster"] = y_kmeans+1
stops

## Determining the best number of clusters

If we use too many clusters, we will have too few datapoints in each cluster and the clusters will not be meaningful. If we have not enough, the data in each of the clusters will be too disparate. There are a number of ways to do this, but for now we will use two of the most common, reliable, and easy to implement methods to determine the optimal number of clusters to describe the data. 

### Elbow plot

An elbow plot shows the number of clusters vs. the distortion of the dataset. The **distortion** is calculated as the weighted sum of squared distances from each data point to its center. Typically, the squared Euclidean distance is used. A higher distortion indicates greater distances *between* the clusters, meaning that they are more distinct (that is good!). 

As a formula, this looks like:

$$\text{Distortion} = \sum_{i=1}^n\frac{1}{m(i)}\sum_{j=1}^{m(i)}(x_j - z_i)^2$$

where $i$ iterates over the number of clusters, $m(i)$ is the number of points in cluster $i$, $j$ iterates over all points assigned to the $i$th cluster, and $z_i$ is the center of cluster $i$. 

This plot shows the average distortion for up to 10 different clusters. What do you notice about the shape of the graph?

In [ ]:
# k means determine k
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(X)
#     kmeanModel.fit(Xx)
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])
# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

Rather than being smooth, the plot is made from distinct segments, and has "elbow shaped" corners. These are the values where there is a significant reduction in distortion as a result of an increase in number of clusters. This is an indication that we should stop dividing the data into further clusters. From this elbow plot, it seems that the optimal number of clusters for this dataset is around 4. 

### Silhouette score
We can also use a metric called a silhouette score, and visualize clustered data projected onto **two** features at a time. The silhouette score measures the degree to which a given datapoint is correctly classified to a given cluster. The silhouette score ranges from [-1,1]. Scores close to -1 indicate that many points are in the wrong cluster, suggesting that there are too few or too many clusters in the model. Scores close to +1 indicate that most points are closely matched to their own clusters, and poorly matched to other clusters. This is what we want, so higher silhouette scores are better.

In [ ]:
y=X

In [ ]:
# experiment by changing the end of the range - no more than 10 should be sufficient.
# Remember that (start, stop) are (inclusive, exclusive)
range_n_clusters = np.arange(2,6)

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(y) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 377 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=377)
    cluster_labels = clusterer.fit_predict(y)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(y, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(y, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.", fontsize=18)
    ax1.set_xlabel("The silhouette coefficient values", fontsize=16)
    ax1.set_ylabel("Cluster label", fontsize=16)

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(y.iloc[:, 0], y.iloc[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.", fontsize=18)
    ax2.set_xlabel("Feature space for the 1st feature", fontsize=16)
    ax2.set_ylabel("Feature space for the 2nd feature", fontsize=16)

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=18, fontweight='bold')

plt.show()

All of these clusters look pretty good! What happens if you keep increasing the number of clusters? A silhouette score above 0.5 is considered a good indication that the data is separable (classifiable into clusters).

## Unsuccessful Clustering

A silhouette score that is a lot less than 0.5 is an indication that the number of clusters is wrong, or that the data is not naturally clusterable. If you get a silhouette score of less than 0.5 with a dataset in the future, consider transforming your data or using a different approach. Here is an example of what your results might look like if your data is not easily clusterable. 

Let's say you have a subset of the `stops` dataset that excludes all population density, Gini index, average monthly shelter costs, and average dwelling value so that you're left only with the `pct_` columns. 

In [ ]:
stops.columns[~stops.columns.str.contains("pct_")]

In [ ]:
y = stops[stops.columns[stops.columns.str.contains("pct_")].values]

In [ ]:
range_n_clusters = [2, 3, 4, 5]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(y) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 377 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=377)
    cluster_labels = clusterer.fit_predict(y)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(y, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(y, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.", fontsize=18)
    ax1.set_xlabel("The silhouette coefficient values", fontsize=16)
    ax1.set_ylabel("Cluster label", fontsize=16)

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(y.iloc[:, 0], y.iloc[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.", fontsize=18)
    ax2.set_xlabel("Feature space for the 1st feature", fontsize=16)
    ax2.set_ylabel("Feature space for the 2nd feature", fontsize=16)

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=18, fontweight='bold')

plt.show()

### Analyzing the Clusters
Let's look at some summary statistics of the 3 clusters and see how they are similar to and different from each other. 

In [ ]:
cluster1 = stops[stops["cluster"] == 1]
cluster2 = stops[stops["cluster"] == 2]
cluster3 = stops[stops["cluster"] == 3]

In [ ]:
print(cluster1.describe())

In [ ]:
print(cluster2.describe())

In [ ]:
print(cluster3.describe())

### Discussion Questions

1. How do each of the clusters differ from each other and how are they the same? Do these differences help you understand the data better for a study or research project? Why or why not?

2. How much and what kind of information was lost when we clustered the transit stops?

3. What are some other applications of clustering related to the urban domain?